# Analysis of decay energies for decay chains and nuclides of importance for decay heat

In [1]:
import os
import urllib
from io import BytesIO
from urllib.request import urlopen, Request
from zipfile import ZipFile
from tempfile import TemporaryDirectory
import shutil
import yaml

import numpy as np
import pandas as pd

In [2]:
import sandy

The values recommended by DDEP are available for comparison at:

|Nuclide  |url                                                          |
|:--------|:------------------------------------------------------------|
| Cs-137  | http://www.nucleide.org/DDEP_WG/Nuclides/Cs-137_tables.pdf  |
| Ba-137m | http://www.nucleide.org/DDEP_WG/Nuclides/Ba-137m_tables.pdf |
| Sr-90   | http://www.nucleide.org/DDEP_WG/Nuclides/Sr-90_tables.pdf   |
| Y-90    | http://www.nucleide.org/DDEP_WG/Nuclides/Y-90_tables.pdf    |
| Ru-106  | http://www.nucleide.org/DDEP_WG/Nuclides/Ru-106_tables.pdf  |
| Rh-106  | http://www.nucleide.org/DDEP_WG/Nuclides/Rh-106_tables.pdf  |
| Ce-144  | http://www.nucleide.org/DDEP_WG/Nuclides/Ce-144_tables.pdf  |
| Pr-144  | http://www.nucleide.org/DDEP_WG/Nuclides/Pr-144_tables.pdf  |
| Eu-154  | http://www.nucleide.org/DDEP_WG/Nuclides/Eu-154_tables.pdf  |
| Cs-134  | http://www.nucleide.org/DDEP_WG/Nuclides/Cs-134_tables.pdf  |
| U-239   | http://www.nucleide.org/DDEP_WG/Nuclides/U-239_tables.pdf   |
| Np-239  | http://www.nucleide.org/DDEP_WG/Nuclides/Np-239_tables.pdf  |
| Pu-238  | http://www.nucleide.org/DDEP_WG/Nuclides/Pu-238_tables.pdf  |
| Pu-239  | http://www.nucleide.org/DDEP_WG/Nuclides/Pu-239_tables.pdf  |
| Pu-240  | http://www.nucleide.org/DDEP_WG/Nuclides/Pu-240_tables.pdf  |
| Am-241  | http://www.nucleide.org/DDEP_WG/Nuclides/Am-241_tables.pdf  |
| Cm-242  | http://www.nucleide.org/DDEP_WG/Nuclides/Cm-242_tables.pdf  |
| Cm-244  | http://www.nucleide.org/DDEP_WG/Nuclides/Cm-244_tables.pdf  |

## Funcions to read decay data using sandy

In [3]:
def read_endfb80_decay_data():
    url = 'https://www.nndc.bnl.gov/endf/b8.0/zips/ENDF-B-VIII.0_decay.zip'
    with TemporaryDirectory(dir=os.getcwd()) as tmpdirname:
        with urlopen(url) as zipresp:
            with ZipFile(BytesIO(zipresp.read())) as zfile:
                zfile.extractall(tmpdirname)

        text = ""
        files = (
            "dec-038_Sr_090.endf",
            "dec-039_Y_090.endf",
            "dec-055_Cs_137.endf",
            "dec-056_Ba_137m1.endf",
            "dec-055_Cs_134.endf",
            "dec-044_Ru_106.endf",
            "dec-045_Rh_106.endf",
            "dec-058_Ce_144.endf",
            "dec-059_Pr_144.endf",
            "dec-063_Eu_154.endf",
            "dec-092_U_239.endf",
            "dec-093_Np_239.endf",
            "dec-094_Pu_238.endf",
            "dec-094_Pu_239.endf",
            "dec-094_Pu_240.endf",
            "dec-095_Am_241.endf",
            "dec-096_Cm_242.endf",
            "dec-096_Cm_244.endf",
        )
        for filename in files:
            file = os.path.join(tmpdirname, "ENDF-B-VIII.0_decay", filename)
            with open(file) as f:
                text += f.read()

    tape = sandy.Endf6.from_text(text)
    rdd = sandy.DecayData.from_endf6(tape, verbose=True)
    return rdd

In [4]:
def read_endfb71_decay_data():
    url = 'https://www.nndc.bnl.gov/endf/b7.1/zips/ENDF-B-VII.1-decay.zip'
    with TemporaryDirectory(dir=os.getcwd()) as tmpdirname:
        with urlopen(url) as zipresp:
            with ZipFile(BytesIO(zipresp.read())) as zfile:
                zfile.extractall(tmpdirname)

        text = ""
        files = (
            "dec-038_Sr_090.endf",
            "dec-039_Y_090.endf",
            "dec-055_Cs_137.endf",
            "dec-056_Ba_137m1.endf",
            "dec-055_Cs_134.endf",
            "dec-044_Ru_106.endf",
            "dec-045_Rh_106.endf",
            "dec-058_Ce_144.endf",
            "dec-059_Pr_144.endf",
            "dec-063_Eu_154.endf",
            "dec-092_U_239.endf",
            "dec-093_Np_239.endf",
            "dec-094_Pu_238.endf",
            "dec-094_Pu_239.endf",
            "dec-094_Pu_240.endf",
            "dec-095_Am_241.endf",
            "dec-096_Cm_242.endf",
            "dec-096_Cm_244.endf",
        )
        for filename in files:
            file = os.path.join(tmpdirname, "decay", filename)
            with open(file) as f:
                text += f.read()

    tape = sandy.Endf6.from_text(text)
    rdd = sandy.DecayData.from_endf6(tape, verbose=True)
    return rdd

In [5]:
def read_endfb70_decay_data():
    url = 'https://www.nndc.bnl.gov/endf/b7.0/zips/dec-ENDF-VII0.endf.zip'
    with TemporaryDirectory(dir=os.getcwd()) as tmpdirname:
        with urlopen(url) as zipresp:
            with ZipFile(BytesIO(zipresp.read())) as zfile:
                zfile.extractall(tmpdirname)

        file = os.path.join(tmpdirname, "dec-ENDF-VII0.endf")
        with open(file) as f:
            text = f.read()

    tape = sandy.Endf6.from_text(text)
    listmat = [
        932,    # Sr-90
        973,    # Y-90
        1782,   # Cs-137
        1831,   # Ba-137m
        1776,   # Cs-134
        1194,   # Ru-106
        1242,   # Rh-106
        1929,   # Pr-144
        1970,   # Pr-144
        2154,   # Eu-154
        3482,   # U-239
        3501,   # Np-239
        3520,   # Pu-238
        3521,   # Pu-239
        3522,   # Pu-240
        3541,   # Am-241
        3563,   # Cm-242
        3565,   # Cm-244
    ]

    rdd = sandy.DecayData.from_endf6(tape.filter_by(listmat=listmat), verbose=True)
    return rdd

In [6]:
def read_endfb68_decay_data():
    # Get decay files from IAEA website
    urlbase = "https://www-nds.iaea.org/public/download-endf/ENDF-B-VI-8/DECAY/mat/"
    files = (
        "endf-b-vi-8_decay_3843.txt", # Sr-90
        "endf-b-vi-8_decay_3928.txt", # Y-90
        "endf-b-vi-8_decay_5537.txt", # Cs-137
        "endf-b-vi-8_decay_5647.txt", # Ba-137m
        "endf-b-vi-8_decay_5528.txt", # Cs-134
        "endf-b-vi-8_decay_4455.txt", # Ru-106
        "endf-b-vi-8_decay_4534.txt", # Rh-106
        "endf-b-vi-8_decay_5849.txt", # Ce-144
        "endf-b-vi-8_decay_5934.txt", # Pr-144
        "endf-b-vi-8_decay_6334.txt", # Eu-154
        "endf-b-vi-8_decay_9240.txt", # U-239
        "endf-b-vi-8_decay_9352.txt", # Np-239
        "endf-b-vi-8_decay_9434.txt", # Pu-238
        "endf-b-vi-8_decay_9437.txt", # Pu-239
        "endf-b-vi-8_decay_9440.txt", # Pu-240
        "endf-b-vi-8_decay_9543.txt", # Am-241
        "endf-b-vi-8_decay_9631.txt", # Cm-242
        "endf-b-vi-8_decay_9637.txt", # Cm-244
    )
    text = ""
    for filename in files:
        url = urlbase + filename
        # set a known browser user agent to ensure access
        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        with urlopen(req) as f:
            text += f.read().decode('utf-8')
    tape = sandy.Endf6.from_text(text)
    rdd = sandy.DecayData.from_endf6(tape, verbose=True)
    return rdd

In [7]:
def read_jeff_decay_data(url):
    with urlopen(url) as f:
        text = f.read().decode('utf-8')
    tape = sandy.Endf6.from_text(text)
    listmat = [
        965,    # Sr-90
        1004,   # Y-90
        1809,   # Cs-137
        1857,   # Ba-137m
        1803,   # Cs-134
        1224,   # Ru-106
        1270,   # Rh-106
        1956,   # Ce-144
        2001,   # Pr-144
        2189,   # Eu-154
        3547,   # U-239
        3566,   # Np-239
        3585,   # Pu-238
        3586,   # Pu-239
        3587,   # Pu-240
        3605,   # Am-241
        3627,   # Cm-242
        3629,   # Cm-244
    ]
    rdd = sandy.DecayData.from_endf6(tape.filter_by(listmat=listmat), verbose=True)
    return rdd

In [8]:
def read_jeff33_decay_data():
    url = "https://www.oecd-nea.org/dbdata/jeff/jeff33/downloads/JEFF33-rdd_all.asc"
    return read_jeff_decay_data(url)

In [9]:
def read_jeff311_decay_data():
    url = "https://www.oecd-nea.org/dbforms/data/eva/evatapes/jeff_31/JEFF31/JEFF31RDD.ASC"
    return read_jeff_decay_data(url)

In [10]:
def read_jeff22_decay_data():
    url = "https://www.oecd-nea.org/dbforms/data/eva/evatapes/jef_22/tape22.asc"
    with urllib.request.urlopen(url) as f:
        text = f.read().decode('utf-8')
    tape = sandy.Endf6.from_text(text)
    listmat = [
        3846,    # Sr-90
        3940,   # Y-90
        5572,   # Cs-137
        5656,   # Ba-137m
        5564,   # Cs-134
        4455,   # Ru-106
        4543,   # Rh-106
        5864,   # Ce-144
        5943,   # Pr-144
        6361,   # Eu-154
        9252,   # U-239
        9352,   # Np-239
        9437,   # Pu-238
        9440,   # Pu-239
        9443,   # Pu-240
        9543,   # Am-241
        9637,   # Cm-242
        9643,   # Cm-244
    ]
    rdd = sandy.DecayData.from_endf6(tape.filter_by(listmat=listmat), verbose=True)
    return rdd

In [11]:
def read_jendl_fpd2011_decay_data():
    urls = {
        "Sr90": "https://wwwndc.jaea.go.jp/cgi-bin/UNZIP_jendl.cgi?/JENDL/JENDL-FPD-2011/038_Sr_090.jfpd11.gz",
        "Y90": "https://wwwndc.jaea.go.jp/cgi-bin/UNZIP_jendl.cgi?/JENDL/JENDL-FPD-2011/039_Y_090.jfpd11.gz",
        "Cs137": "https://wwwndc.jaea.go.jp/cgi-bin/UNZIP_jendl.cgi?/JENDL/JENDL-FPD-2011/055_Cs_137.jfpd11.gz",
        "Ba137m": "https://wwwndc.jaea.go.jp/cgi-bin/UNZIP_jendl.cgi?/JENDL/JENDL-FPD-2011/056_Ba_137M.jfpd11.gz",
        "Cs134": "https://wwwndc.jaea.go.jp/cgi-bin/UNZIP_jendl.cgi?/JENDL/JENDL-FPD-2011/055_Cs_134.jfpd11.gz",
        "Ru106": "https://wwwndc.jaea.go.jp/cgi-bin/UNZIP_jendl.cgi?/JENDL/JENDL-FPD-2011/044_Ru_106.jfpd11.gz",
        "Rh106": "https://wwwndc.jaea.go.jp/cgi-bin/UNZIP_jendl.cgi?/JENDL/JENDL-FPD-2011/045_Rh_106.jfpd11.gz",
        "Ce144": "https://wwwndc.jaea.go.jp/cgi-bin/UNZIP_jendl.cgi?/JENDL/JENDL-FPD-2011/058_Ce_144.jfpd11.gz",
        "Pr144": "https://wwwndc.jaea.go.jp/cgi-bin/UNZIP_jendl.cgi?/JENDL/JENDL-FPD-2011/059_Pr_144.jfpd11.gz",
        "Eu154": "https://wwwndc.jaea.go.jp/cgi-bin/UNZIP_jendl.cgi?/JENDL/JENDL-FPD-2011/063_Eu_154.jfpd11.gz",
        "U239": "https://wwwndc.jaea.go.jp/cgi-bin/UNZIP_jendl.cgi?/JENDL/JENDL-DDF-2015/092_U_239.jddf15.gz",
        "Np239": "https://wwwndc.jaea.go.jp/cgi-bin/UNZIP_jendl.cgi?/JENDL/JENDL-DDF-2015/093_Np_239.jddf15.gz",
        "Pu238": "https://wwwndc.jaea.go.jp/cgi-bin/UNZIP_jendl.cgi?/JENDL/JENDL-DDF-2015/094_Pu_238.jddf15.gz",
        "Pu239": "https://wwwndc.jaea.go.jp/cgi-bin/UNZIP_jendl.cgi?/JENDL/JENDL-DDF-2015/094_Pu_239.jddf15.gz",
        "Pu240": "https://wwwndc.jaea.go.jp/cgi-bin/UNZIP_jendl.cgi?/JENDL/JENDL-DDF-2015/094_Pu_240.jddf15.gz",
        "Am241": "https://wwwndc.jaea.go.jp/cgi-bin/UNZIP_jendl.cgi?/JENDL/JENDL-DDF-2015/095_Am_241.jddf15.gz",
        "Cm242": "https://wwwndc.jaea.go.jp/cgi-bin/UNZIP_jendl.cgi?/JENDL/JENDL-DDF-2015/096_Cm_242.jddf15.gz",
        "Cm244": "https://wwwndc.jaea.go.jp/cgi-bin/UNZIP_jendl.cgi?/JENDL/JENDL-DDF-2015/096_Cm_244.jddf15.gz",
    }
    text = ""
    for url in urls.values():
        with urlopen(url) as f:
            text += f.read().decode('utf-8')
    tape = sandy.Endf6.from_text(text)
    rdd = sandy.DecayData.from_endf6(tape, verbose=True)
    return rdd

Decay data are stored as `sandy.DecayData` objects into a dictionary instance called `rdds`. 

In [12]:
rdds = {
    "ENDF/B-VI.8": read_endfb68_decay_data(),
    "ENDF/B-VII.0": read_endfb70_decay_data(),
    "ENDF/B-VII.1": read_endfb71_decay_data(),
    "ENDF/B-VIII.0": read_endfb80_decay_data(),
    "JEFF-2.2": read_jeff22_decay_data(),
    "JEFF-3.1.1": read_jeff311_decay_data(),
    "JEFF-3.3": read_jeff33_decay_data(),
    "JENDL/FPD-2011": read_jendl_fpd2011_decay_data(),
}

INFO:  reading 'ZAM=380900'...
INFO:  reading 'ZAM=390900'...
INFO:  reading 'ZAM=441060'...
INFO:  reading 'ZAM=451060'...
INFO:  reading 'ZAM=551340'...
INFO:  reading 'ZAM=551370'...
INFO:  reading 'ZAM=561371'...
INFO:  reading 'ZAM=581440'...
INFO:  reading 'ZAM=591440'...
INFO:  reading 'ZAM=631540'...
INFO:  reading 'ZAM=922390'...
INFO:  reading 'ZAM=932390'...
INFO:  reading 'ZAM=942380'...
INFO:  reading 'ZAM=942390'...
INFO:  reading 'ZAM=942400'...
INFO:  reading 'ZAM=952410'...
INFO:  reading 'ZAM=962420'...
INFO:  reading 'ZAM=962440'...
INFO:  reading 'ZAM=380900'...
INFO:  reading 'ZAM=390900'...
INFO:  reading 'ZAM=441060'...
INFO:  reading 'ZAM=451060'...
INFO:  reading 'ZAM=551340'...
INFO:  reading 'ZAM=551370'...
INFO:  reading 'ZAM=561371'...
INFO:  reading 'ZAM=581440'...
INFO:  reading 'ZAM=591440'...
INFO:  reading 'ZAM=631540'...
INFO:  reading 'ZAM=922390'...
INFO:  reading 'ZAM=932390'...
INFO:  reading 'ZAM=942380'...
INFO:  reading 'ZAM=942390'...
INFO:  r

## Data analysis

Let's store the requested information into a dataframe for further processing

In [13]:
entries = []
for lib, rdd in rdds.items():
    for nuclide in rdd.data:
        for decay in ("alpha", "beta", "gamma", "total"):
            energy = rdd.data[nuclide]['decay_energy'][decay]
            uncertainty = rdd.data[nuclide]['decay_energy_uncertainties'][decay]
            entry = {
                "nuclide": sandy.zam2latex(nuclide),
                "decay": decay,
                "energy": energy * 1e-3,            # in keV
                "uncertainty": uncertainty * 1e-3,  # in keV
                "library": lib,
            }
            entries.append(entry)
data = pd.DataFrame(entries)

In [14]:
data

,nuclide,decay,energy,uncertainty,library
0,$^{90}$Sr,alpha,0.00000e+00,0.00000e+00,ENDF/B-VI.8
1,$^{90}$Sr,beta,1.95900e+02,7.00000e-01,ENDF/B-VI.8
2,$^{90}$Sr,gamma,0.00000e+00,0.00000e+00,ENDF/B-VI.8
3,$^{90}$Sr,total,1.95900e+02,7.00000e-01,ENDF/B-VI.8
4,$^{90}$Y,alpha,0.00000e+00,0.00000e+00,ENDF/B-VI.8
...,...,...,...,...,...
571,$^{144}$Pr,total,1.23790e+03,5.01597e+00,JENDL/FPD-2011
572,$^{154}$Eu,alpha,0.00000e+00,0.00000e+00,JENDL/FPD-2011
573,$^{154}$Eu,beta,2.72000e+02,1.00000e+01,JENDL/FPD-2011
574,$^{154}$Eu,gamma,1.24300e+03,6.00000e+00,JENDL/FPD-2011


In [15]:
def get_table(data, values):
    dfs = {}

    decay = "total"
    dfs[decay] = data.query(f"decay == '{decay}'").pivot_table(index="library", columns="nuclide", values=values)
    dfs[decay].columns = pd.MultiIndex.from_product([[decay], dfs[decay].columns], names=["decay", "nuclide"])

    decay = "alpha"
    dfs[decay] = data.query(f"decay == '{decay}'").pivot_table(index="library", columns="nuclide", values=values)
    dfs[decay].columns = pd.MultiIndex.from_product([[decay], dfs[decay].columns], names=["decay", "nuclide"])

    decay = "beta"
    dfs[decay] = data.query(f"decay == '{decay}'").pivot_table(index="library", columns="nuclide", values=values)
    dfs[decay].columns = pd.MultiIndex.from_product([[decay], dfs[decay].columns], names=["decay", "nuclide"])

    decay = "gamma"
    dfs[decay] = data.query(f"decay == '{decay}'").pivot_table(index="library", columns="nuclide", values=values)
    dfs[decay].columns = pd.MultiIndex.from_product([[decay], dfs[decay].columns], names=["decay", "nuclide"])

    return dfs["alpha"].merge(dfs["beta"], left_index=True, right_index=True)\
                       .merge(dfs["gamma"], left_index=True, right_index=True)\
                       .merge(dfs["total"], left_index=True, right_index=True)

### Mean recoverable energy values (in keV) per library, nuclide and decay mode

In [16]:
energies = get_table(data, "energy").T
print(energies.to_string())

library            ENDF/B-VI.8  ENDF/B-VII.0  ENDF/B-VII.1  ENDF/B-VIII.0    JEFF-2.2  JEFF-3.1.1    JEFF-3.3  JENDL/FPD-2011
decay nuclide                                                                                                                
alpha $^{106}$Rh   0.00000e+00   0.00000e+00   0.00000e+00    0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00     0.00000e+00
      $^{106}$Ru   0.00000e+00   0.00000e+00   0.00000e+00    0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00     0.00000e+00
      $^{134}$Cs   0.00000e+00   0.00000e+00   0.00000e+00    0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00     0.00000e+00
      $^{137m}$Ba  0.00000e+00   0.00000e+00   0.00000e+00    0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00     0.00000e+00
      $^{137}$Cs   0.00000e+00   0.00000e+00   0.00000e+00    0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00     0.00000e+00
      $^{144}$Ce   0.00000e+00   0.00000e+00   0.00000e+00    0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00     0.00

### Uncertainty on the mean recoverable energy values (in keV) per library, nuclide and decay mode

In absolute units (keV).

In [17]:
uncert = get_table(data, "uncertainty").T
print(uncert.to_string())

library            ENDF/B-VI.8  ENDF/B-VII.0  ENDF/B-VII.1  ENDF/B-VIII.0    JEFF-2.2  JEFF-3.1.1    JEFF-3.3  JENDL/FPD-2011
decay nuclide                                                                                                                
alpha $^{106}$Rh   0.00000e+00   0.00000e+00   0.00000e+00    0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00     0.00000e+00
      $^{106}$Ru   0.00000e+00   0.00000e+00   0.00000e+00    0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00     0.00000e+00
      $^{134}$Cs   0.00000e+00   0.00000e+00   0.00000e+00    0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00     0.00000e+00
      $^{137m}$Ba  0.00000e+00   0.00000e+00   0.00000e+00    0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00     0.00000e+00
      $^{137}$Cs   0.00000e+00   0.00000e+00   0.00000e+00    0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00     0.00000e+00
      $^{144}$Ce   0.00000e+00   0.00000e+00   0.00000e+00    0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00     0.00

... and in relative units (%).

In [18]:
print((uncert / energies * 100).fillna(0).to_string())

library            ENDF/B-VI.8  ENDF/B-VII.0  ENDF/B-VII.1  ENDF/B-VIII.0    JEFF-2.2  JEFF-3.1.1    JEFF-3.3  JENDL/FPD-2011
decay nuclide                                                                                                                
alpha $^{106}$Rh   0.00000e+00   0.00000e+00   0.00000e+00    0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00     0.00000e+00
      $^{106}$Ru   0.00000e+00   0.00000e+00   0.00000e+00    0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00     0.00000e+00
      $^{134}$Cs   0.00000e+00   0.00000e+00   0.00000e+00    0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00     0.00000e+00
      $^{137m}$Ba  0.00000e+00   0.00000e+00   0.00000e+00    0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00     0.00000e+00
      $^{137}$Cs   0.00000e+00   0.00000e+00   0.00000e+00    0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00     0.00000e+00
      $^{144}$Ce   0.00000e+00   0.00000e+00   0.00000e+00    0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00     0.00

## Mean recoverable energy per decay chain

In [19]:
outputs = []
for lib in energies.columns:

    parent = 551370
    daughter = 561371
    chain = "$^{137}$Cs/$^{137m}$Ba"
    mode = '10x0' # decay mode: '10' = beta + gamma
    br = rdds[lib].data[parent]["decay_modes"][mode]['branching_ratio']
    for decay in ("gamma", "beta", "total"):
        out = {
            "decay": decay,
            "energy": rdds[lib].data[parent]['decay_energy'][decay] * 1e-3 +\
                      rdds[lib].data[daughter]['decay_energy'][decay] * 1e-3 * br,
            "library": lib,
            "chain": chain,
        }
        outputs.append(out)

    parent = 380900
    daughter = 390900
    chain = "$^{90}$Sr/$^{90}$Y"
    mode = '10x0' # decay mode: '10' = beta + gamma
    br = rdds[lib].data[parent]["decay_modes"][mode]['branching_ratio']
    for decay in ("gamma", "beta", "total"):
        out = {
            "decay": decay,
            "energy": rdds[lib].data[parent]['decay_energy'][decay] * 1e-3 +\
                      rdds[lib].data[daughter]['decay_energy'][decay] * 1e-3 * br,
            "library": lib,
            "chain": chain,
        }
        outputs.append(out)

    parent = 441060
    daughter = 451060
    chain = "$^{106}$Ru/$^{106}$Rh"
    mode = '10x0' # decay mode: '10' = beta + gamma
    br = rdds[lib].data[parent]["decay_modes"][mode]['branching_ratio']
    for decay in ("gamma", "beta", "total"):
        out = {
            "decay": decay,
            "energy": rdds[lib].data[parent]['decay_energy'][decay] * 1e-3 +\
                      rdds[lib].data[daughter]['decay_energy'][decay] * 1e-3 * br,
            "library": lib,
            "chain": chain,
        }
        outputs.append(out)

    parent = 581440
    daughter = 591440
    chain = "$^{144}$Ce/$^{144}$Pr"
    mode = '10x0' # decay mode: '10' = beta + gamma
    br = rdds[lib].data[parent]["decay_modes"][mode]['branching_ratio']
    for decay in ("gamma", "beta", "total"):
        out = {
            "decay": decay,
            "energy": rdds[lib].data[parent]['decay_energy'][decay] * 1e-3 +\
                      rdds[lib].data[daughter]['decay_energy'][decay] * 1e-3 * br,
            "library": lib,
            "chain": chain,
        }
        outputs.append(out)

chain_energies = pd.DataFrame(outputs).pivot_table(index=("decay", "chain"), columns="library", values="energy")
chain_energies

library                       ENDF/B-VI.8  ENDF/B-VII.0  ENDF/B-VII.1  \
decay chain                                                             
beta  $^{106}$Ru/$^{106}$Rh   1.42103e+03   1.42086e+03   1.42346e+03   
      $^{137}$Cs/$^{137m}$Ba  1.91094e+02   1.83449e+02   1.82846e+02   
      $^{144}$Ce/$^{144}$Pr   1.28338e+03   1.28812e+03   1.28833e+03   
      $^{90}$Sr/$^{90}$Y      1.12990e+03   1.12941e+03   1.12881e+03   
gamma $^{106}$Ru/$^{106}$Rh   2.06000e+02   2.06071e+02   2.06072e+02   
      $^{137}$Cs/$^{137m}$Ba  3.33643e+01   3.35260e+01   3.16593e+01   
      $^{144}$Ce/$^{144}$Pr   4.74954e+01   4.77972e+01   4.78332e+01   
      $^{90}$Sr/$^{90}$Y      1.70000e-03   3.06074e-05   8.85767e-04   
total $^{106}$Ru/$^{106}$Rh   1.62703e+03   1.62693e+03   1.62953e+03   
      $^{137}$Cs/$^{137m}$Ba  2.24458e+02   2.16975e+02   2.14505e+02   
      $^{144}$Ce/$^{144}$Pr   1.33087e+03   1.33592e+03   1.33616e+03   
      $^{90}$Sr/$^{90}$Y      1.12990e+03   1.12941e+03   1.12882e+03   

library                       ENDF/B-VIII.0    JEFF-2.2  JEFF-3.1.1  \
decay chain                                                           
beta  $^{106}$Ru/$^{106}$Rh     1.42346e+03 1.42254e+03 1.42149e+03   
      $^{137}$Cs/$^{137m}$Ba    1.82846e+02 1.89492e+02 1.91272e+02   
      $^{144}$Ce/$^{144}$Pr     1.28833e+03 1.28148e+03 1.27530e+03   
      $^{90}$Sr/$^{90}$Y        1.12881e+03 1.12964e+03 1.10782e+03   
gamma $^{106}$Ru/$^{106}$Rh     2.06072e+02 2.08660e+02 2.04290e+02   
      $^{137}$Cs/$^{137m}$Ba    3.16593e+01 3.22973e+01 3.35149e+01   
      $^{144}$Ce/$^{144}$Pr     4.78332e+01 4.78665e+01 5.23492e+01   
      $^{90}$Sr/$^{90}$Y        8.85767e-04 3.10000e-04 1.23656e-03   
total $^{106}$Ru/$^{106}$Rh     1.62953e+03 1.63120e+03 1.62578e+03   
      $^{137}$Cs/$^{137m}$Ba    2.14505e+02 2.21790e+02 2.24787e+02   
      $^{144}$Ce/$^{144}$Pr     1.33616e+03 1.32935e+03 1.32765e+03   
      $^{90}$Sr/$^{90}$Y        1.12882e+03 1.12964e+03 1.10782e+03   

library                         JEFF-3.3  JENDL/FPD-2011  
decay chain                                               
beta  $^{106}$Ru/$^{106}$Rh  1.42248e+03     1.42003e+03  
      $^{137}$Cs/$^{137m}$Ba 1.83296e+02     1.90473e+02  
      $^{144}$Ce/$^{144}$Pr  1.27961e+03     1.29684e+03  
      $^{90}$Sr/$^{90}$Y     1.12732e+03     1.12960e+03  
gamma $^{106}$Ru/$^{106}$Rh  2.05096e+02     2.06000e+02  
      $^{137}$Cs/$^{137m}$Ba 3.37545e+01     3.15244e+01  
      $^{144}$Ce/$^{144}$Pr  4.88705e+01     4.81220e+01  
      $^{90}$Sr/$^{90}$Y     1.23656e-03     1.25000e-03  
total $^{106}$Ru/$^{106}$Rh  1.62758e+03     1.62603e+03  
      $^{137}$Cs/$^{137m}$Ba 2.17050e+02     2.21997e+02  
      $^{144}$Ce/$^{144}$Pr  1.32848e+03     1.34496e+03  
      $^{90}$Sr/$^{90}$Y     1.12732e+03     1.12960e+03

### Uncertainty propagation

Create a dataframe with linear sensitivities and uncertainties on both branching ratios and recoverable energies.

In [20]:
uncertainties = {}
sensitivities = {}

for lib in energies.columns:

    parent = 551370
    daughter = 561371
    chain = "$^{137}$Cs/$^{137m}$Ba"
    mode = '10x0' # decay mode: '10' = beta + gamma
    for decay in ("gamma", "beta"):
        uncertainties[(lib, decay, chain, "br")] = rdds[lib].data[parent]["decay_modes"][mode]['branching_ratio_uncertainty']
        uncertainties[(lib, decay, chain, "energy_parent")] = rdds[lib].data[parent]['decay_energy_uncertainties'][decay]
        uncertainties[(lib, decay, chain, "energy_daughter")] = rdds[lib].data[daughter]['decay_energy_uncertainties'][decay]
        sensitivities[(lib, decay, chain, "br")] = rdds[lib].data[daughter]['decay_energy'][decay] * 1e-3
        sensitivities[(lib, decay, chain, "energy_parent")] = 1 * 1e-3
        sensitivities[(lib, decay, chain, "energy_daughter")] = rdds[lib].data[parent]["decay_modes"][mode]['branching_ratio'] * 1e-3

    parent = 380900
    daughter = 390900
    chain = "$^{90}$Sr/$^{90}$Y"
    mode = '10x0' # decay mode: '10' = beta + gamma
    for decay in ("gamma", "beta"):
        uncertainties[(lib, decay, chain, "br")] = rdds[lib].data[parent]["decay_modes"][mode]['branching_ratio_uncertainty']
        uncertainties[(lib, decay, chain, "energy_parent")] = rdds[lib].data[parent]['decay_energy_uncertainties'][decay]
        uncertainties[(lib, decay, chain, "energy_daughter")] = rdds[lib].data[daughter]['decay_energy_uncertainties'][decay]
        sensitivities[(lib, decay, chain, "br")] = rdds[lib].data[daughter]['decay_energy'][decay] * 1e-3
        sensitivities[(lib, decay, chain, "energy_parent")] = 1 * 1e-3
        sensitivities[(lib, decay, chain, "energy_daughter")] = rdds[lib].data[parent]["decay_modes"][mode]['branching_ratio'] * 1e-3

    parent = 441060
    daughter = 451060
    chain = "$^{106}$Ru/$^{106}$Rh"
    mode = '10x0' # decay mode: '10' = beta + gamma
    for decay in ("gamma", "beta"):
        uncertainties[(lib, decay, chain, "br")] = rdds[lib].data[parent]["decay_modes"][mode]['branching_ratio_uncertainty']
        uncertainties[(lib, decay, chain, "energy_parent")] = rdds[lib].data[parent]['decay_energy_uncertainties'][decay]
        uncertainties[(lib, decay, chain, "energy_daughter")] = rdds[lib].data[daughter]['decay_energy_uncertainties'][decay]
        sensitivities[(lib, decay, chain, "br")] = rdds[lib].data[daughter]['decay_energy'][decay] * 1e-3
        sensitivities[(lib, decay, chain, "energy_parent")] = 1 * 1e-3
        sensitivities[(lib, decay, chain, "energy_daughter")] = rdds[lib].data[parent]["decay_modes"][mode]['branching_ratio'] * 1e-3

    parent = 581440
    daughter = 591440
    chain = "$^{144}$Ce/$^{144}$Pr"
    mode = '10x0' # decay mode: '10' = beta + gamma
    for decay in ("gamma", "beta"):
        uncertainties[(lib, decay, chain, "br")] = rdds[lib].data[parent]["decay_modes"][mode]['branching_ratio_uncertainty']
        uncertainties[(lib, decay, chain, "energy_parent")] = rdds[lib].data[parent]['decay_energy_uncertainties'][decay]
        uncertainties[(lib, decay, chain, "energy_daughter")] = rdds[lib].data[daughter]['decay_energy_uncertainties'][decay]
        sensitivities[(lib, decay, chain, "br")] = rdds[lib].data[daughter]['decay_energy'][decay] * 1e-3
        sensitivities[(lib, decay, chain, "energy_parent")] = 1 * 1e-3
        sensitivities[(lib, decay, chain, "energy_daughter")] = rdds[lib].data[parent]["decay_modes"][mode]['branching_ratio'] * 1e-3

In [21]:
data_unc = pd.DataFrame([sensitivities, uncertainties], index=["sensitivity", "uncertainty"]).T
data_unc.index = pd.MultiIndex.from_tuples(data_unc.index, names=["library", "decay", "chain", "parameter"])
data_unc.reset_index(inplace=True)

Propagate the uncertainties to the beta and gamma mean recoverable energies for the chain.

In [22]:
outputs = []
for (lib, chain, decay), df in data_unc.groupby(["library", "chain", "decay"]):
    variance = (df.sensitivity * df.uncertainty).apply(lambda x: x**2).sum()
    out = {
        "variance": variance,
        "uncertainty": np.sqrt(variance),
        "library": lib,
        "decay": decay,
        "chain": chain,
    }
    outputs.append(out)
uncertainties = pd.DataFrame(outputs)
uncertainties

,variance,uncertainty,library,decay,chain
0,1.60064e+01,4.00080e+00,ENDF/B-VI.8,beta,$^{106}$Ru/$^{106}$Rh
1,1.60000e+01,4.00000e+00,ENDF/B-VI.8,gamma,$^{106}$Ru/$^{106}$Rh
2,3.35831e-02,1.83257e-01,ENDF/B-VI.8,beta,$^{137}$Cs/$^{137m}$Ba
3,5.32279e-02,2.30712e-01,ENDF/B-VI.8,gamma,$^{137}$Cs/$^{137m}$Ba
4,7.60572e+00,2.75785e+00,ENDF/B-VI.8,beta,$^{144}$Ce/$^{144}$Pr
...,...,...,...,...,...
59,1.41796e+00,1.19078e+00,JENDL/FPD-2011,gamma,$^{137}$Cs/$^{137m}$Ba
60,2.57391e+01,5.07337e+00,JENDL/FPD-2011,beta,$^{144}$Ce/$^{144}$Pr
61,2.49271e-01,4.99270e-01,JENDL/FPD-2011,gamma,$^{144}$Ce/$^{144}$Pr
62,2.08000e+00,1.44222e+00,JENDL/FPD-2011,beta,$^{90}$Sr/$^{90}$Y


Propagate the uncertainties to the total recoverable energy for the chains.

In [23]:
series = uncertainties[["library", "chain", "variance"]].groupby(["library", "chain"]).sum()
total_uncertainties = series.assign(decay="total", uncertainty=np.sqrt(series.variance)).reset_index()
total_uncertainties

,library,chain,variance,decay,uncertainty
0,ENDF/B-VI.8,$^{106}$Ru/$^{106}$Rh,3.20064e+01,total,5.65742e+00
1,ENDF/B-VI.8,$^{137}$Cs/$^{137m}$Ba,8.68109e-02,total,2.94637e-01
2,ENDF/B-VI.8,$^{144}$Ce/$^{144}$Pr,8.44544e+00,total,2.90610e+00
3,ENDF/B-VI.8,$^{90}$Sr/$^{90}$Y,1.49000e+00,total,1.22066e+00
4,ENDF/B-VII.0,$^{106}$Ru/$^{106}$Rh,1.48858e+02,total,1.22007e+01
5,ENDF/B-VII.0,$^{137}$Cs/$^{137m}$Ba,1.87528e+00,total,1.36941e+00
6,ENDF/B-VII.0,$^{144}$Ce/$^{144}$Pr,2.45543e+01,total,4.95523e+00
7,ENDF/B-VII.0,$^{90}$Sr/$^{90}$Y,1.77624e-04,total,1.33276e-02
8,ENDF/B-VII.1,$^{106}$Ru/$^{106}$Rh,1.53043e+02,total,1.23711e+01
9,ENDF/B-VII.1,$^{137}$Cs/$^{137m}$Ba,1.86566e+00,total,1.36589e+00


In absolute units (keV)...

In [24]:
absolute_unc = pd.concat([uncertainties, total_uncertainties])\
                 .pivot_table(index=["decay", "chain"], columns="library", values="uncertainty")
absolute_unc

library                       ENDF/B-VI.8  ENDF/B-VII.0  ENDF/B-VII.1  \
decay chain                                                             
beta  $^{106}$Ru/$^{106}$Rh   4.00080e+00   1.19215e+01   1.20957e+01   
      $^{137}$Cs/$^{137m}$Ba  1.83257e-01   6.64268e-01   6.60619e-01   
      $^{144}$Ce/$^{144}$Pr   2.75785e+00   4.93530e+00   5.05936e+00   
      $^{90}$Sr/$^{90}$Y      1.22066e+00   1.33276e-02   1.08363e+00   
gamma $^{106}$Ru/$^{106}$Rh   4.00000e+00   2.59545e+00   2.59545e+00   
      $^{137}$Cs/$^{137m}$Ba  2.30712e-01   1.19751e+00   1.19551e+00   
      $^{144}$Ce/$^{144}$Pr   9.16361e-01   4.44036e-01   4.77801e-01   
      $^{90}$Sr/$^{90}$Y      3.00000e-04   6.55873e-06   6.72030e-05   
total $^{106}$Ru/$^{106}$Rh   5.65742e+00   1.22007e+01   1.23711e+01   
      $^{137}$Cs/$^{137m}$Ba  2.94637e-01   1.36941e+00   1.36589e+00   
      $^{144}$Ce/$^{144}$Pr   2.90610e+00   4.95523e+00   5.08187e+00   
      $^{90}$Sr/$^{90}$Y      1.22066e+00   1.33276e-02   1.08363e+00   

library                       ENDF/B-VIII.0    JEFF-2.2  JEFF-3.1.1  \
decay chain                                                           
beta  $^{106}$Ru/$^{106}$Rh     1.20957e+01 4.59467e+01 1.23837e+01   
      $^{137}$Cs/$^{137m}$Ba    6.60619e-01 1.39395e+00 9.09934e-01   
      $^{144}$Ce/$^{144}$Pr     5.05936e+00 4.98955e+00 8.51990e+00   
      $^{90}$Sr/$^{90}$Y        1.08363e+00 7.16617e-01 1.30014e+00   
gamma $^{106}$Ru/$^{106}$Rh     2.59545e+00 6.05060e+00 2.84718e+00   
      $^{137}$Cs/$^{137m}$Ba    1.19551e+00 1.79794e+00 7.42155e-02   
      $^{144}$Ce/$^{144}$Pr     4.77801e-01 4.95213e-01 6.82250e-01   
      $^{90}$Sr/$^{90}$Y        6.72030e-05 0.00000e+00 1.02541e-04   
total $^{106}$Ru/$^{106}$Rh     1.23711e+01 4.63434e+01 1.27068e+01   
      $^{137}$Cs/$^{137m}$Ba    1.36589e+00 2.27501e+00 9.12955e-01   
      $^{144}$Ce/$^{144}$Pr     5.08187e+00 5.01407e+00 8.54717e+00   
      $^{90}$Sr/$^{90}$Y        1.08363e+00 7.16617e-01 1.30014e+00   

library                         JEFF-3.3  JENDL/FPD-2011  
decay chain                                               
beta  $^{106}$Ru/$^{106}$Rh  5.78487e+00     1.30002e+01  
      $^{137}$Cs/$^{137m}$Ba 9.41252e-01     9.08956e-01  
      $^{144}$Ce/$^{144}$Pr  1.15358e+00     5.07337e+00  
      $^{90}$Sr/$^{90}$Y     1.30014e+00     1.44222e+00  
gamma $^{106}$Ru/$^{106}$Rh  1.54133e+00     3.00000e+00  
      $^{137}$Cs/$^{137m}$Ba 7.47459e-02     1.19078e+00  
      $^{144}$Ce/$^{144}$Pr  5.61751e-01     4.99270e-01  
      $^{90}$Sr/$^{90}$Y     1.02541e-04     1.00000e-04  
total $^{106}$Ru/$^{106}$Rh  5.98668e+00     1.33419e+01  
      $^{137}$Cs/$^{137m}$Ba 9.44215e-01     1.49805e+00  
      $^{144}$Ce/$^{144}$Pr  1.28309e+00     5.09787e+00  
      $^{90}$Sr/$^{90}$Y     1.30014e+00     1.44222e+00

... and in relative units (%).

In [25]:
relative_unc = absolute_unc / chain_energies * 100
relative_unc

library                       ENDF/B-VI.8  ENDF/B-VII.0  ENDF/B-VII.1  \
decay chain                                                             
beta  $^{106}$Ru/$^{106}$Rh   2.81542e-01   8.39032e-01   8.49745e-01   
      $^{137}$Cs/$^{137m}$Ba  9.58990e-02   3.62099e-01   3.61298e-01   
      $^{144}$Ce/$^{144}$Pr   2.14890e-01   3.83140e-01   3.92707e-01   
      $^{90}$Sr/$^{90}$Y      1.08032e-01   1.18004e-03   9.59969e-02   
gamma $^{106}$Ru/$^{106}$Rh   1.94175e+00   1.25949e+00   1.25948e+00   
      $^{137}$Cs/$^{137m}$Ba  6.91492e-01   3.57188e+00   3.77617e+00   
      $^{144}$Ce/$^{144}$Pr   1.92937e+00   9.29001e-01   9.98888e-01   
      $^{90}$Sr/$^{90}$Y      1.76471e+01   2.14286e+01   7.58698e+00   
total $^{106}$Ru/$^{106}$Rh   3.47715e-01   7.49923e-01   7.59181e-01   
      $^{137}$Cs/$^{137m}$Ba  1.31266e-01   6.31135e-01   6.36764e-01   
      $^{144}$Ce/$^{144}$Pr   2.18361e-01   3.70924e-01   3.80333e-01   
      $^{90}$Sr/$^{90}$Y      1.08032e-01   1.18004e-03   9.59968e-02   

library                       ENDF/B-VIII.0    JEFF-2.2  JEFF-3.1.1  \
decay chain                                                           
beta  $^{106}$Ru/$^{106}$Rh     8.49745e-01 3.22992e+00 8.71178e-01   
      $^{137}$Cs/$^{137m}$Ba    3.61298e-01 7.35623e-01 4.75727e-01   
      $^{144}$Ce/$^{144}$Pr     3.92707e-01 3.89359e-01 6.68068e-01   
      $^{90}$Sr/$^{90}$Y        9.59969e-02 6.34378e-02 1.17361e-01   
gamma $^{106}$Ru/$^{106}$Rh     1.25948e+00 2.89974e+00 1.39370e+00   
      $^{137}$Cs/$^{137m}$Ba    3.77617e+00 5.56685e+00 2.21440e-01   
      $^{144}$Ce/$^{144}$Pr     9.98888e-01 1.03457e+00 1.30327e+00   
      $^{90}$Sr/$^{90}$Y        7.58698e+00 0.00000e+00 8.29244e+00   
total $^{106}$Ru/$^{106}$Rh     7.59181e-01 2.84107e+00 7.81581e-01   
      $^{137}$Cs/$^{137m}$Ba    6.36764e-01 1.02575e+00 4.06142e-01   
      $^{144}$Ce/$^{144}$Pr     3.80333e-01 3.77183e-01 6.43780e-01   
      $^{90}$Sr/$^{90}$Y        9.59968e-02 6.34378e-02 1.17360e-01   

library                         JEFF-3.3  JENDL/FPD-2011  
decay chain                                               
beta  $^{106}$Ru/$^{106}$Rh  4.06675e-01     9.15491e-01  
      $^{137}$Cs/$^{137m}$Ba 5.13516e-01     4.77210e-01  
      $^{144}$Ce/$^{144}$Pr  9.01511e-02     3.91211e-01  
      $^{90}$Sr/$^{90}$Y     1.15331e-01     1.27675e-01  
gamma $^{106}$Ru/$^{106}$Rh  7.51516e-01     1.45631e+00  
      $^{137}$Cs/$^{137m}$Ba 2.21440e-01     3.77733e+00  
      $^{144}$Ce/$^{144}$Pr  1.14947e+00     1.03751e+00  
      $^{90}$Sr/$^{90}$Y     8.29244e+00     8.00000e+00  
total $^{106}$Ru/$^{106}$Rh  3.67828e-01     8.20520e-01  
      $^{137}$Cs/$^{137m}$Ba 4.35022e-01     6.74806e-01  
      $^{144}$Ce/$^{144}$Pr  9.65833e-02     3.79036e-01  
      $^{90}$Sr/$^{90}$Y     1.15330e-01     1.27675e-01